# Introduction to Classification

In this module, we will explore how to use R for binary classification problems.

If you are unfamiliar with notebooks, please review some basics [here](https://github.com/michhar/useR2016-tutorial-jupyter). 

## Essential Tips

A very brief summary of the critical components and commands within jupyter are:

1. Critically, press `Ctrl+Enter` to run (or render) the current cell.
2. Output will print to the notebook. You may have to scroll up to see it all.
3. Get help for any function by typing a question mark and then its name into
   the console: `?rxLinMod`. It will split the window, and will bring up the documentation for 
   that function below.
5. Files will appear in the specified directory. You can find them by selecting File in the menu bar and selecting "Open...". This will open a new browser window with a file navigator.
6. R objects can be viewed by typing `ls()` in an R cell.
7. Run all the example code!

There are a number of hands-on exercises in the document, so while you can run the notebook from beginning to end, you will get a lot more out of it by actually walking through cell-by-cell, and filling out the corresponding exercises.

These notebooks are based on a tutorial presented at a Microsoft conference in June of 2016. The original files are available [here](https://github.com/joseph-rickert/MLADS_JUNE_2016).



## Introduction

This script presents a binary classification problem using a cell segmentation data set that is included in the `caret` package. The data is described in the paper: Hill et al "Impact of image segmentation on high-content screening data quality for SK-BR-3 cells" BMC fioinformatics (2007) vol 8 (1) pp. 340

"Well-segmented"" cells are cells for which location and size may be accurrately detremined through optical measurements. Cells that are not Well-segmented (WS) are said to be "Poorly-segmented"" (PS). Given a set of optical measurements can we predict which cells will be PS? 

Before we get started, we'll source a configuration file in the next cell, load some required packages, and write a quick function to help with splitting u. It simply makes sure that the relevant R packages and datasets are available. You do not need to look at it, but if you are interested, you can view the configuration file [here](Resources/config.R), and the source code for the `splitData()` function [here](Resources/splitData.R). It may take a few moments to run the first time you run it, but it should be fast afterwards.

In [ ]:
source("Resources/config.R")  # source the configuration file to make sure all packages are installed and datasets are available.
#library(rattle)       # visualize Tree
#library(ROCR)         # Plot the ROC Curve

source("Resources/splitData.R") # Function to divide data into training, and test sets 
set.seed(42)           # Set seed for simulation repeatability

## Read the Data and Prepare the Training and Test Sets

Let's start by looking at the data. The data is in the `caret` package, and we can start by simply looking at the help for the dataset, which gives us the data dictionary.

In [ ]:
library(caret)
?segmentationData   # view documentation which is effectively the data dictionary.

## Process the data

Next, we'll load the data with `data()`, create a processed data.frame called `dF` that has fewer columns, and then remove the full `segmentationData` object so we just have access to the data set with fewer columns. Specifically we use **negative indices** to drop elements - in this particular case, we use negative indices to drop columns.

In [ ]:
data(segmentationData)
dF <- segmentationData[,-c(1,2)]  #Remove non-predictive variables
head(dF,2)  ## view the first couple of rows


## Training and Testing

Overfitting is a serious issue. In order to try to evaluate over-fitting, we will create a subset of data that we can estimate (or train) a model on, and then a second subset of data that we can evaluate (or test) the model. We wrote a function called `splitData()` (source code available [here](Resources/splitData.R)) that returns a two element list, where the first element within the list is the set of row indices appropriate for training, and the second element within the list is the set of row indices appropriate for testing.

One note - remember that to access an element within a list, we use double-square brackets (`[[`).

First, before we do that step, let's see the function. To see the function, we can type the name of the function without adding parentheses:

In [ ]:
splitData

## Using splitData()

Now let's use it!

In [ ]:
indices <- splitData(dF)       # Generate indidices to split data
trainInd <- indices[[1]] # Training data row indices. Note use of `[[` to extract the first element of the list
testInd <- indices[[2]]  # Test data row indices. Note use of `[[` to extract the second element of the list
length(trainInd)
length(testInd)
dim(dF[trainInd,])
dim(dF[testInd,])
nrow(dF) == (length(trainInd) + length(testInd))

## Explore the Data

Before we actually start modeling, let's do some simple statistics.

First, start with some univariate statistics:

In [ ]:
sapply(dF,class)
summary(dF)


## Bivariate

Next, let's look at the correlations among the independent variables.

In [ ]:
library(corrplot)
corrplot(cor(dF[,-1]), 
         method="ellipse",
         tl.pos="n",
         title="Correlation of segmentationData Predictor Variables")

## Build a Tree Model with rpart   

The `rpart` algorithm based on recursive partitioning       
(See section 11.2 of [*Data Mining with Rattle and R* by Williams](http://www.springer.com/us/book/9781441998897))   

Here's a brief pseudo-code outline of the steps in the rpart algorithm:

1. Partition the data set according to some criterion of "best" partition   
2. Take each of the two newly created subsets, and start over with step 1.
3. Once a partition is made, stick with it (i.e. use a greedy approach)   
4. Continue until a stopping criterion is met.
 
In order to do this, we must define the measure of the 'best' partition.

Two common measures are:
1. Information gain (the default)   
2. Gini   

Information gain can be defined by the following steps:

For all possible splits (partitions)   
1. Split data, $D$, into to subsets $S_1$ and $S_2$ where $D = S_1 \cup S_2$   
2. Calculate impurity $I_1$ and $I_2$ associated with $S_1$ and $S_2$, such that $I_s = \sum_k -p_k log p_k$, where $p_k$ is the probability of class $k$ within partition $s$. This measure is frequently called entropy in information theory (and is frequently represented as $H$.
3. The gain in information is the change in the entropy of $D$ when you don't know anything about the partition split $s$ and the entropy of $D$ after you know what split you're in: $Information = I(D) - \sum p_s I_s $, where $p_s$ and $I_s$ correspond to the probability and impurity of split $s$ respectively.

Once you calculate that information gain for all possible splits, you simply select the split with greatest information gain.

Let's try it out. We will start by loading the library, and then we will set up a control data structure to govern things such as stopping criteria and complexity parameters. 

In [ ]:
library(rpart)
ctrl <- rpart.control(minsplit = 20,      # Min obs at node for splitting
                      cp = 0.05,          # Complexity parameter
                      maxcompete = 4,     # Num. competor splits in retained
                      maxsurrogate = 5,   # Num. surrogate splits retained
                      usesurrogate = 2,   # How to use surrogate splits
                      xval = 10,          # Num. of cross validations
                      surrogatestyle = 0, # How to select surrogates
                      maxdepth = 30       # Max depth of tree
)

## Fit the tree   

Now that we have specified the the control structure, we can actually estimate the model using the `rpart()` function. We use a `formula` to specify the model, we explicitly state that the only data used in the fitting process should be the training data, and we explicitly state the control structure.

Specifically in this case, the formula `Class ~ .` is used to create the specification. A formula is typically a two-sided structure, with a tilde (`~`) separating out the two sides. The outcome variable is indicated on the left-hand side of the formula, and predictor variables are specified on the right-hand side of the formula. It is useful to read the `~` as "as a function of". So, the formula can be read as predict `Class` "as a function of" all other variables in the dataset. The `.` in the formula represents all other variables in the dataset.

In [ ]:
form <- formula(Class~ .)				# Describe the model to R
tree_fit <- rpart(formula=form,
              data=dF[trainInd,],
              control = ctrl)	# Build the model

## Viewing and Interpreting the Model Results     

Now that we have estimated it - we can take a first look at it by simply typing the name of the object. When we do that, we will get some information about the structure of the estimated partitions.

Specifically, every line of the output that describes the tree structure will have:

1. `node`: a node number
2. `split`: the logic for how the node splits the data   
3. `n`: the number of observations considered at that split    
4. `loss`: the number of incorrectly classified observations   
5. `yval`: the majority class at that node   
6. `yprob`: the distribution of classes at that node   
 


In [ ]:
tree_fit

In [ ]:
# a <- print(tree_fit)
print(tree_fit$frame[2,])


 So for the second line above: Pressure3pm>=1011.9 204 16 No (0.92156863 0.07843137)   
  1. node: 2)      
  2. split: if ToralIntenCh2 < 47253.5 go left down tree   
  3. n: 659 obversations went down this branch      
  4. loss: 57 misclassified observations   
  5. Most observations were PS   
  6. 91% of obs have target var PS, 9% are WS   

## Examine the results    

If we want to get even more information, we can use `summary()` on the fit object.

In [ ]:
summary(tree_fit)             # Summary of rpart tree model

## Plot the tree

It is also very common to visualize the tree. One of the benefits of a recursive partitioning algorithm is that the splits provide for relatively easy-to-interpret break points. There are a few packages for making interactive plots, but this simply uses the `plot.rpart()` function:

In [ ]:
plot(tree_fit)
text(tree_fit)

## Evaluate model performance on the test set 

Next, we might want to evaluate performance on the test set. The best way to do this is to generate predictions on the testing data, and then use those predictions to calculate confusion matrices and other metrics such as AUC.

In [ ]:
## calculate!
pr <- predict(tree_fit, dF[testInd, ], type="class")

## Calculate the Confusion Matrix

We can use functions from other packages to simplify the creation of a confusion matrix:

In [ ]:
response <- dF[testInd,1] # Actual responses
confusionMatrix(data=pr,reference=response)      

## Draw the ROC Curve

In [ ]:
## create a different type of prediction!
prob <- predict(tree_fit, dF[testInd, ], type="prob")

library(ROCR)
pred <- prediction(prob[,1],response,label.ordering=c("WS","PS"))
perf <- performance(pred, measure = "tpr", x.measure = "fpr") 
plot(perf, col=rainbow(10),
     xlim = c(0,1),
     ylim = c(0,1),
     xlab="False Positive Rate",
     ylab="True Positive Rate",
     main="ROC Curve")

### Explore an unpruned tree

The complexity parameter sets the minimum benefit that must be gained at each split of the decision tree. (default = .01) 

Because we are performing some cross-validation during estimation, we can examine the effect of cp on other folds while we estimate. Typical behavior on this cross-validation error with cp=0 is to see the error reate decrease at first and then begin to increase.

In [ ]:
ctrl2 <- rpart.control(minsplit = 0,  # Min obs at node for splitting
                       cp = 0,     # Complexity parameter
                       maxcompete = 4, # No. competor splits in retained
                       maxsurrogate = 5, # No. surrogate splits retained
                       usesurrogate = 2, # How to use surrogate splits
                       xval = 10,        # No. of cross validations
                       surrogatestyle = 0, # How to select surrogates
                       maxdepth = 30       # Max depth of tree
)

## Now, just estimate the model with the new control structure!

In [ ]:
big_tree <- rpart(formula=form,control=ctrl2,data=dF[trainInd,])

## Now we can imagine more!

In [ ]:
print(big_tree$cptable)

In [ ]:
plotcp(big_tree)